#  Librerías

### Librerías pesadas
Para ejecutar solo una vez

In [1]:
import math
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np
import tqdm

/mnt/shared/Documents/proyecto-recsys/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Librerías livianas
Para ejecutar múltiples veces

# Configuración General

Variables relacionadas al procesamiento de datos y del modelo en sí

### Variables de Preprocesamiento

In [2]:

# Porcentaje para usar solo una fracción del dataset de usuario.
# si al eliminar usuarios quedan viajes o POI sin visitas, estos también
# serán eliminados
USER_FRAC = 0.4
MIN_POI_VISITS = 5
MAX_SEQUENCES_PER_USER = 100
SEQUENCE_LENGTH = 14

In [3]:
BATCH_SIZE=64
EPOCHS=100

In [4]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 70

# Gowalla Dataset

In [5]:
! ./download-gowalla.sh

Already Downloaded


### Cargar Datos

In [6]:
users    = pd.read_csv('download/gowalla/gowalla_userinfo.csv')
friends  = pd.read_csv('download/gowalla/gowalla_friendship.csv')
checkins = pd.read_csv('download/gowalla/gowalla_checkins.csv')
pois_1   = pd.read_csv('download/gowalla/gowalla_spots_subset1.csv', encoding='iso-8859-1')
pois_2   = pd.read_csv('download/gowalla/gowalla_spots_subset2.csv', encoding='iso-8859-1')
pois     = pd.concat((pois_1, pois_2), ignore_index=True)


# Preprocesamiento

### Usuarios

Revisamos la distrubución de checkins de usuarios

In [7]:
users.sample(5)

,id,bookmarked_spots_count,challenge_pin_count,country_pin_count,highlights_count,items_count,photos_count,pins_count,province_pin_count,region_pin_count,state_pin_count,trips_count,friends_count,stamps_count,checkin_num,places_num
34656,75899,0,12,1,0,9,26,20,0,8,7,0,5,120,215,120
86661,182616,0,7,2,0,10,2,9,0,2,0,0,2,69,187,69
150114,346572,0,1,2,0,4,0,3,0,2,0,0,2,2,2,2
171904,499371,0,1,1,0,2,0,2,0,1,0,0,1,3,3,3
134279,304452,0,13,2,0,10,76,15,0,2,0,0,22,145,338,145


In [8]:
users.checkin_num.describe()

count    407533.000000
mean         88.341212
std         435.982581
min           0.000000
25%           1.000000
50%          10.000000
75%          52.000000
max       46981.000000
Name: checkin_num, dtype: float64

nos quedamos con un porcentaje de los usuarios y los filtramos los usuarios segun los checkins que tengan

In [9]:
print('Current users', len(users))
users = users.sample(frac=USER_FRAC)
users = users[users.checkin_num >= SEQUENCE_LENGTH]
users = users[['id']]
print('Reduced users', len(users))

Current users 407533
Reduced users 74054


In [10]:
checkins[checkins.userid ==  69996]

,userid,placeid,datetime
4223526,69996,269432,2011-06-01T19:10:19Z
4223527,69996,61280,2011-05-31T20:35:45Z
4223528,69996,49812,2011-05-30T23:29:12Z
4223529,69996,205779,2011-05-30T23:28:53Z
4223530,69996,6511613,2011-05-30T23:28:25Z
...,...,...,...
4224301,69996,282847,2009-12-28T12:48:22Z
4224302,69996,117306,2009-12-28T12:45:56Z
4224303,69996,150947,2009-12-27T22:37:39Z
4224304,69996,261322,2009-12-27T22:27:05Z


### Amigos

In [11]:
friends.sample(5)

,userid1,userid2
3474735,1558665,552472
1511286,125822,190972
605786,31465,71697
35607,1184,16646
3311006,2545141,326250


### Checkins

In [12]:
checkins.sample(5)

,userid,placeid,datetime
13138162,55634,245262,2009-12-21T11:33:23Z
2822518,8360,64310,2009-11-06T23:58:23Z
21454266,307075,380427,2010-06-13T17:37:05Z
22462055,349890,14554,2010-08-10T13:56:30Z
31106130,265211,1481524,2010-07-22T12:10:47Z


Eliminamos los checkins de los usuarios no sampleados

In [13]:
print('Current checkins', len(checkins))
checkins = pd.merge(checkins, users, how='inner', left_on='userid', right_on='id', copy=False)[checkins.columns]
checkins = checkins.reset_index(drop=True)
print('Reduced checkins', len(checkins))

Current checkins 36001959
Reduced checkins 14335594


### POIS

In [14]:
pois.sample(5)

,id,created_at,lng,lat,photos_count,checkins_count,users_count,radius_meters,highlights_count,items_count,max_items_count,spot_categories,name,city_state,Unnamed: 5,Unnamed: 6
1623442,4454761,2010-09-26T03:03:38Z,-90.394769,38.780422,1.0,4.0,3.0,75.0,0.0,1.0,10.0,"[{'url': '/categories/47', 'name': 'Modern Hot...",NaN,NaN,NaN,NaN
1957207,6565272,2010-12-10T13:00:23Z,132.464818,34.455441,0.0,7.0,1.0,75.0,0.0,0.0,10.0,"[{'url': '/categories/106', 'name': 'Grocery'}]",NaN,NaN,NaN,NaN
1390156,1594631,2010-08-11T00:25:03Z,-86.750305,34.751688,0.0,2.0,2.0,75.0,0.0,0.0,10.0,"[{'url': '/categories/154', 'name': 'Burgers'}]",NaN,NaN,NaN,NaN
116522,133706,2009-11-26T06:59:19Z,8.626318,52.322112,0.0,8.0,4.0,75.0,0.0,1.0,10.0,"[{'url': '/categories/119', 'name': 'Gas & Aut...",NaN,NaN,NaN,NaN
443971,495915,2010-02-02T20:02:02Z,-122.872520,38.608496,0.0,5.0,3.0,75.0,0.0,1.0,10.0,"[{'url': '/categories/454', 'name': 'Subway'}]",NaN,NaN,NaN,NaN


Filtramos los pois si han sido visitadas pocas veces según los parámetros que definimos

In [15]:
visited_pois = pd.merge(pois, checkins, left_on='id', right_on='placeid', how='inner', copy=False)

In [16]:
visited_pois['visited_count'] = np.zeros(len(visited_pois))

visited_pois = visited_pois[['id', 'visited_count']].groupby(by='id').count()
visited_pois = visited_pois[visited_pois.visited_count >= MIN_POI_VISITS]

pois = pd.merge(pois, visited_pois, on='id', how='inner', copy=False)

Nos quedamos con sólo las columnas que nos importan

In [17]:
pois = pois[['id', 'lat', 'lng', 'visited_count']]
pois 

,id,lat,lng,visited_count
0,8904,39.052318,-94.607499,88
1,8932,32.927662,-97.254356,28
2,8936,39.053318,-94.591995,17
3,8938,39.052824,-94.590311,130
4,8947,37.331880,-122.029631,1069
...,...,...,...,...
642599,7527671,27.390431,-82.511700,5
642600,7527865,30.335072,-87.134716,5
642601,7529658,35.403926,-97.727437,5
642602,7533476,35.847316,-78.805891,6


Ahora eliminamos los checkins de pois que ya no existen

In [18]:
print('Current checkins', len(checkins))
checkins = pd.merge(pois, checkins, left_on='id', right_on='placeid', how='inner', copy=False)[checkins.columns]
checkins = checkins.reset_index(drop=True)
print('Reduced checkins', len(checkins))

Current checkins 14335594
Reduced checkins 11745848


Finalmente eliminamos nuevamente a los usuarios que se quedaron sin suficientes checkins

In [19]:
print('Current users', len(users))
users = pd.merge(checkins, users, how='inner', left_on='userid', right_on='id', copy=False)[users.columns].drop_duplicates()
print('Reduced users', len(users))

Current users 74054
Reduced users 73971


### Reasignación de IDs

In [20]:
users = users.reset_index(drop=True)
users['user_sid'] = users.index

pois = pois.reset_index(drop=True)
pois['place_sid'] = pois.index

### Agregar Datos

Crearemos un dataset unificado que usaremos para entrenar el modelo de los embeddings

In [21]:
pois.head()

,id,lat,lng,visited_count,place_sid
0,8904,39.052318,-94.607499,88,0
1,8932,32.927662,-97.254356,28,1
2,8936,39.053318,-94.591995,17,2
3,8938,39.052824,-94.590311,130,3
4,8947,37.331880,-122.029631,1069,4


In [22]:
users.head()

,id,user_sid
0,1,0
1,151,1
2,159,2
3,181,3
4,50372,4


In [23]:
checkins.head()

,userid,placeid,datetime
0,1,8904,2009-09-02T19:35:39Z
1,1,8904,2009-09-02T08:06:01Z
2,1,8904,2009-09-02T08:03:20Z
3,1,8904,2009-09-02T08:02:37Z
4,1,8904,2009-09-02T06:39:42Z


In [24]:
users_checkins = pd.merge(users, checkins, left_on='id', right_on='userid', copy=False).drop('id', axis=1)
users_checkins = pd.merge(users_checkins, pois[['id', 'place_sid']], left_on='placeid', right_on='id', copy=False)
users_checkins['date'] = pd.to_datetime(users_checkins['datetime'])
users_checkins = users_checkins.drop('datetime', axis=1)
users_checkins.sort_values(by=['user_sid', 'date'], inplace=True)

In [25]:
users_checkins[['user_sid', 'place_sid']].head(20)

,user_sid,place_sid
71,0,0
3609,0,28
1738,0,10
3632,0,31
3631,0,31
70,0,0
69,0,0
3608,0,28
68,0,0
67,0,0


In [26]:
users_checkins[['user_sid', 'place_sid']].tail(20)

,user_sid,place_sid
11745833,73970,642451
11745891,73970,642452
11745832,73970,642451
11745890,73970,642452
11745831,73970,642451
11745889,73970,642452
11745830,73970,642451
11745888,73970,642452
11745829,73970,642451
11745828,73970,642451


Podemos ver que hay un problema: Muchas veces se repiten los POI consecutivos de un usuario, sin embargo eso no aporta mucha información al embedding, por lo que los eliminaremos en el siguiente paso

In [27]:
users_checkins['last_place_sid'] = users_checkins['place_sid'].shift(1)
user_checkins = users_checkins[users_checkins.place_sid != users_checkins.last_place_sid]
user_checkins = user_checkins.drop('last_place_sid', axis=1)

In [28]:
user_checkins.tail()

,user_sid,userid,placeid,id,place_sid,date
11745825,73970,2615815,7361926,7361926,642451,2011-07-02 02:33:01+00:00
11745885,73970,2615815,7361973,7361973,642452,2011-07-02 11:38:43+00:00
11745824,73970,2615815,7361926,7361926,642451,2011-07-03 02:46:12+00:00
11745884,73970,2615815,7361973,7361973,642452,2011-07-03 10:53:14+00:00
11745823,73970,2615815,7361926,7361926,642451,2011-07-03 23:47:06+00:00


In [29]:
from collections import defaultdict

In [30]:
user_poi_seq = defaultdict(lambda: [])
user_date_seq = defaultdict(lambda: [])

In [31]:
for user_sid, place_sid, date in zip(users_checkins['user_sid'], users_checkins['place_sid'], users_checkins['date']):
    user_poi_seq[user_sid].append(place_sid)
    user_date_seq[user_sid].append(date)

Generamos secuencias de puntos de interes visitados por usuarios de un largo predefinido para entrenar el modelo de embeddings

In [32]:
from random import sample

poi_sequence_dataset = []

for user_sid, sequence in user_poi_seq.items():
    if len(sequence) < SEQUENCE_LENGTH: continue

    candidate_indexes = list(range(0, len(sequence) - SEQUENCE_LENGTH, SEQUENCE_LENGTH))

    n_sequences = min(len(candidate_indexes), MAX_SEQUENCES_PER_USER)
    start_indexes = sample(candidate_indexes, n_sequences)

    for idx in start_indexes:
        new_seq = sequence[idx:idx + SEQUENCE_LENGTH]
        poi_sequence_dataset.append(new_seq) 
    

In [33]:
def split_list(input, frac=0.5):
    split_index = int(len(input) * frac)
    return input[:split_index], input[split_index:]

## Split de Datos

Realizamos separacion en train / test / split de 80 / 10 / 10

In [34]:
train_poi_sequence, rest = split_list(poi_sequence_dataset, 0.8)
test_poi_sequence, val_poi_sequence = split_list(rest)

In [35]:
def split_history_target(sequences):
    history = [ seq[:-1] for seq in sequences]
    targets = [ seq[-1] for seq in sequences]
    return history, targets

In [36]:
train_seq_history, train_seq_target = split_history_target(train_poi_sequence)
test_seq_history, test_seq_target = split_history_target(test_poi_sequence)
val_seq_history, val_seq_target = split_history_target(val_poi_sequence)

In [37]:
unique_pois = { poi for sequence in  poi_sequence_dataset  for poi in sequence }

print("Total pois incluidos en dataset:", len(unique_pois))
print("Porcentaje de POIs que se usaran en el modelo",  len(unique_pois) * 100 / len(pois))

Total pois incluidos en dataset: 637525
Porcentaje de POIs que se usaran en el modelo 99.2096221000803


In [38]:
print("Total Train", len(train_seq_history))
print("Total Test ", len(test_seq_history))
print("Total Val  ", len(val_seq_history))

Total Train 538191
Total Test  67274
Total Val   67274


Ya tenemos nuestros datos listos para entrenar!

# Modelo

In [39]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size=None, emb_dim=None, hidden_dim=None, sample_length=None):
        super(EmbeddingModel, self).__init__()
        
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.hidden = nn.Linear(sample_length * emb_dim, hidden_dim)
        self.hidden_activation = nn.ReLU()
        self.output = nn.Linear(hidden_dim, vocab_size)
        self.output_activation = nn.LogSoftmax(dim=-1)

    def forward(self, xs):
        batch_size = xs.size()[0]

        # embed and merge
        xs = self.emb(xs)
        xs = torch.reshape(xs, (batch_size, -1))

        # hidden layer
        hidden = self.hidden(xs)
        hidden = self.hidden_activation(hidden)

        # output log probabilities
        output_logits = self.output(hidden)
        output_log_probs = self.output_activation(output_logits)
        
        return output_log_probs

    def predict(self, xs):
        return torch.argmax(self.forward(xs))

In [40]:
class Trainer:
    def __init__(self, model):
        pass

In [41]:
from torch.utils.data import DataLoader

def dataset_to_tensors(sequences, targets):
    return [[torch.tensor(x), torch.tensor(y)] for x, y in zip(sequences, targets)]


train_tensors = dataset_to_tensors(train_seq_history, train_seq_target)
train_dataloader = DataLoader(train_tensors, batch_size=BATCH_SIZE, shuffle=True)

test_tensors = dataset_to_tensors(test_seq_history, test_seq_target)
test_dataloader = DataLoader(test_tensors, batch_size=BATCH_SIZE, shuffle=True)

val_tensors = dataset_to_tensors(val_seq_history, val_seq_target)
val_dataloader = DataLoader(val_tensors, batch_size=BATCH_SIZE, shuffle=True)

In [42]:

# Globals
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss = nn.NLLLoss()

# Training Parameters
print_every = 100
test_every = 1000 # 2000
adjust_every = 2500 # 500
epsilon_0 = 1e-2
r = 1e-8

# Tensorboard-related
# experiment_name = Path('act-{}_emb-{}_hidden-{}_sample-{}_batch-{}_optim-{}_lr-{}_adjust-every-{}'.format(
#     'relu',
#     emb_dim,
#     hidden_dim,
#     sample_length,
#     batch_size,
#     'sgd',
#     epsilon_0,
#     adjust_every
# ))
# experiment_path = Path('run') / experiment_name
# writer = SummaryWriter(experiment_path)

# Get the dataset

# Create the model
print("Initializing model ...")
lm = EmbeddingModel(
    vocab_size=len(pois),
    emb_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    sample_length=SEQUENCE_LENGTH - 1
)


lm.to(device)
optimizer = optim.SGD(lm.parameters(), lr=epsilon_0) # optim.Adam(lm.parameters(), amsgrad=True)

losses = []

# train_iter = iter(train_dataloader)

print("Beginning training ...")
i = 0
for epoch in range(EPOCHS):    
    # Zero the parameter gradients
    print("Epoch", epoch + 1)
    for xs, ys in train_dataloader:
        optimizer.zero_grad()
        
        xs, ys = xs.to(device), ys.to(device)

        # Get the averaged batch loss
        output = loss(lm(xs), ys)

        # Run the backward pass (calculate gradients)
        output.backward()

        # Update the model
        optimizer.step()

        # Save and print statistics
        losses.append(output.item())

        if i % print_every == (print_every - 1):
            # Get average loss
            avg_loss = sum(losses) / len(losses)
            losses = []

            # Compute perplexity
            perp = math.exp(avg_loss)

            # Print and log the train loss
            print("({}) : {}".format((i + 1), perp))
            # writer.add_scalar('Training Loss', perp, i)

        if i % test_every == (test_every - 1):
            print("\nEvaluating model on test set ...")
            
            lm.eval()

            with torch.no_grad():
                test_iter = iter(test_dataloader)

                test_losses = []            
                for xs, ys in tqdm.tqdm(test_iter, total=len(test_iter)):
                    xs, ys = xs.to(device), ys.to(device)
                    output = loss(lm(xs), ys)
                    test_losses.append(output.item())


            avg_test_loss = sum(test_losses) / len(test_losses)
            test_losses = []

            # Compute perplexity
            test_perp = math.exp(avg_test_loss)

            # Print and log the test loss
            print("Average Test Loss: {}\n".format(test_perp))
            # writer.add_scalar('Test Loss', test_perp, i)
                    
            lm.train()

        # Update the learning rate every iteration, just as in the paper
        if i % adjust_every == (adjust_every - 1):
            for g in optimizer.param_groups:
                g['lr'] = epsilon_0 / (1 + r * (i / adjust_every))

        i += 1

Initializing model ...
Beginning training ...
Epoch 1
(100) : 664621.8204858219
(200) : 666678.4938409738
(300) : 659306.1553012102
(400) : 658983.6534558545
(500) : 662998.3803865968
(600) : 664290.0221633972
(700) : 660475.1497861792
(800) : 661754.6055563579
(900) : 661922.4548294584
(1000) : 663151.0187936182

Evaluating model on test set ...


100%|██████████| 1052/1052 [00:09<00:00, 105.44it/s]


Average Test Loss: 660356.1465816613

(1100) : 661065.7229027775
(1200) : 660436.174074024
(1300) : 658825.2262301946
(1400) : 664649.4878616733
(1500) : 659256.2270944153
(1600) : 661304.7979805643
(1700) : 661041.9493850174
(1800) : 658560.6751194334
(1900) : 658923.2427703885
(2000) : 655093.0827973963

Evaluating model on test set ...


100%|██████████| 1052/1052 [00:09<00:00, 105.91it/s]


Average Test Loss: 659100.0056358612

(2100) : 658465.8271960105
(2200) : 657702.3669865049
(2300) : 658383.5379250979
(2400) : 658645.1473040022
(2500) : 654364.9221400875
(2600) : 660079.2766535
(2700) : 658707.0150247839
(2800) : 658661.8746971502
(2900) : 655432.2136554641
(3000) : 652130.7854586433

Evaluating model on test set ...


...

Average Test Loss: 652110.0461389056

(13100) : 635162.1012412953
(13200) : 635264.8063277359
(13300) : 633027.870646415
(13400) : 632195.3514828363
(13500) : 632743.1923925142
(13600) : 631227.3159367021
(13700) : 628764.7268590624
(13800) : 634771.5568326795
(13900) : 636329.2113861404
(14000) : 629481.6352342826

Evaluating model on test set ...


100%|██████████| 1052/1052 [00:09<00:00, 106.14it/s]


Average Test Loss: 651934.5891421415

(14100) : 633142.4154319244
(14200) : 634504.4471665344
(14300) : 633540.549050934
(14400) : 633201.2052644313
(14500) : 631651.773926282
(14600) : 635361.3717649934


: 

: 

In [ ]:
def accuracy(model, test_seqs, test_targets):
    x = torch.tensor(test_seq)
    y
        
    return corrects / len(test_seqs)